## Setup the Simulator

In [ ]:
%load_ext autoreload
%autoreload 2


import os
import re
import json
import glob
import random
import numpy as np
import shutil
import sys
sys.path.append('simulation')
from unity_simulator.comm_unity import UnityCommunication
from dataset_utils import execute_script_utils as utils
from evolving_graph import scripts

def setup():
    comm = UnityCommunication()
    return comm

def process_file_inbuilt(script_file, graph_file):
    return utils.render_script_from_path(setup(),
                                script_file, graph_file,
                                {"processing_time_limit": 500, "image_width": 320, "image_height": 240, "image_synthesis": ['normal'], "gen_vid": True, "file_name_prefix": "test", "camera_mode": 'PERSON_TOP'}, scene_id=1)

In [ ]:
bare_graph_file = 'example_graphs/CustomSceneBare_graph.json'
init_graph_file = 'example_graphs/CustomScene_graph.json'
unnecessary_nodes = ['floor','wall','ceiling','window','character','door','doorjamb']

## Add necessary objects

In [ ]:
with open (bare_graph_file,'r') as f:
    graph_dict = json.load(f)

nodes = {n['id']:n['class_name'] for n in graph_dict['nodes']}

for e in graph_dict['edges']:
    rel = e['relation_type']
    if rel != "CLOSE" and rel!= "FACING":
        n1 = e['from_id']
        n2 = e['to_id']
        if nodes[n1] not in unnecessary_nodes and nodes[n2] not in unnecessary_nodes:
            print(f'{nodes[n1]}({n1}) {rel} {nodes[n2]}({n2})')

new_obj_id = 1000
with open ('resources/object_states.json','r') as f:
    object_states = json.load(f)
with open ('resources/properties_data.json','r') as f:
    object_properties = json.load(f)

def add_to_scene(obj, relation, parent_id):
    assert(relation in ["INSIDE","ON"])
    global new_obj_id, graph_dict
    new_node = None
    new_edges = []
    # graph_dict['nodes'].append({"id": new_obj_id, "class_name": obj, "category": "placable_objects", "properties": object_properties[obj], "states": object_states[obj], "prefab_name": None, "bounding_box": None})
    graph_dict['nodes'].append({"id": new_obj_id, "class_name": obj, "category": "placable_objects", "properties": [], "states": [], "prefab_name": None, "bounding_box": None})
    graph_dict['edges'].append({"from_id":new_obj_id, "relation_type":relation, "to_id":parent_id})
    for e in graph_dict['edges']:
        if e is not None:
            if e['from_id'] == parent_id and e['relation_type'] in ["INSIDE","ON"]:
                ne = e.copy()
                ne.update({"from_id":new_obj_id})
                graph_dict['edges'].append(ne)
    new_obj_id += 1

In [ ]:
add_to_scene("food_apple","ON",230)

In [ ]:
with open (init_graph_file,'w') as f:
    json.dump(graph_dict, f)

## Prepare guide based on graph

In [ ]:
# import networkx as nx
edges = {}

# graph = nx.DiGraph()
with open (init_graph_file,'r') as f:
    graph_dict = json.load(f)
nodes = {n['id']:n['class_name'] for n in graph_dict['nodes']}
nodes_by_room = {n['class_name']:{} for n in graph_dict['nodes'] if n['category'] == "Rooms"}
node_rooms = {n['class_name']:{} for n in graph_dict['nodes'] if n['category'] == "Rooms"}
node_map = {'<'+n['class_name']+'>': '<'+n['class_name']+'> ('+str(n['id'])+')' for n in graph_dict['nodes'] if n['category'] == "Rooms"}

for e in graph_dict['edges']:
    rel = e['relation_type']
    if rel != "CLOSE" and rel!= "FACING":
        n1 = e['from_id']
        n2 = e['to_id']
        if nodes[n1] not in unnecessary_nodes and nodes[n2] not in unnecessary_nodes:
            edge_id = (nodes[n1],rel,nodes[n2])
            if edge_id not in edges:
                edges[edge_id] = e.update({'from_class':nodes[n1], 'to_class':nodes[n2]})
            if nodes[n2] in nodes_by_room:
                nodes_by_room[nodes[n2]][n1] = nodes[n1]
                node_rooms[n1] = nodes[n2]
            # graph.add_edge(f'{nodes[n1]}({n1})', f'{nodes[n2]}({n2})', type=rel)

print(len(nodes))
print(sum([len(l) for l in nodes_by_room.values()]))
print([nodes[id] for id in node_rooms if node_rooms[id]==None])


In [ ]:
usable_nodes_by_room = {}
for room,nodelist in nodes_by_room.items():
    usable_nodes_by_room[room] = {}
    for id, name in nodelist.items():
        if name not in usable_nodes_by_room[room]:
            usable_nodes_by_room[room][name] = id

repeated_nodes = []
for l in usable_nodes_by_room.values():
    repeated_nodes += list(l.keys())
repeated_nodes  = [n for n in repeated_nodes if repeated_nodes.count(n)>1]

expanded_nodes_by_room = {}
for room,nodelist in usable_nodes_by_room.items():
    expanded_nodes_by_room[room] = {}
    for name, id in nodelist.items():
        full_name = name
        if name in repeated_nodes:
            full_name += '_'+room
        expanded_nodes_by_room[room][full_name] = (name,id)

for l in expanded_nodes_by_room.values():
    node_map.update({f'<{key}>':f'<{val[0]}> ({val[1]})' for key,val in l.items()})

print(node_map.keys())


In [ ]:
from evolving_graph.scripts import Script, parse_script_line
from evolving_graph.environment import EnvironmentGraph
from evolving_graph.execution import ScriptExecutor
import evolving_graph.utils as utils

class ProgramExecutor():
    def __init__(self, program_file='program.txt', graph_file=init_graph_file):
        with open (graph_file,'r') as f:
            self.init_graph_dict = json.load(f)
        self.init_graph = EnvironmentGraph(self.init_graph_dict)
        self.read_program(program_file)

    def read_program(self, file_name):
        self.action_scripts = []
        self.single_script = []
        self.action_headers = []
        with open(file_name) as f:
            lines = []
            index = 1
            for line in f:
                if line.startswith('##'):
                    self.action_headers.append(line[2:].strip())
                    self.action_scripts.append(lines)
                    lines = []
                    index = 1
                if '[' not in line:
                    continue
                line = line.strip()
                if len(line) > 0 and not line.startswith('#'):
                    mapped_line = line
                    for full_name, name_id in node_map.items():
                        mapped_line = mapped_line.replace(full_name, name_id)
                    scr_line = parse_script_line(mapped_line, index, custom_patt_params = r'\<(.+?)\>\s*\((.+?)\)')
                    print(scr_line)
                    self.single_script.append(scr_line)
                    lines.append(scr_line)
                    index += 1

    def execute(self):
        name_equivalence = utils.load_name_equivalence()
        executor = ScriptExecutor(self.init_graph, name_equivalence)
        graphs = []
        for header, script in zip(self.action_headers, self.action_scripts):
            success, state, _ = executor.execute(Script(script), w_graph_list=False)
            if not success:
                raise RuntimeError(f'Execution of {header} failed')
            graphs.append(state._graph)
        return self.action_headers, graphs

In [ ]:
ProgramExecutor().execute()
